In [71]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Carga de las bases de datos
df_2004 = pd.read_stata('usu_individual_T104.dta', convert_categoricals=False) 
df_2024 = pd.read_excel('usu_individual_T124.xlsx')

# Unificamos nombres de columnas a mayúsculas
df_2004.columns = df_2004.columns.str.upper()
df_2024.columns = df_2024.columns.str.upper()

# Obtenemos columnas comunes para trabajar con los mismos datos
columnas_comunes = df_2004.columns.intersection(df_2024.columns)

# Filtramos ambas bases por esas columnas
df_2004_filtrado = df_2004[columnas_comunes]
df_2024_filtrado = df_2024[columnas_comunes]

# Combinamos las bases
df_combinado = pd.concat([df_2004_filtrado, df_2024_filtrado], ignore_index=True)

df_eph = df_combinado

df_eph.to_excel('EPH_2004_2024.xlsx', index=False)

In [72]:
variables_a_limpiar = ['PP3E_TOT', 'PP3F_TOT']

df_eph[variables_a_limpiar] = df_eph[variables_a_limpiar].replace([99, 999, 9999], np.nan)

# Variable 'horastrab'
df_eph['horastrab'] = df_eph['PP3E_TOT'] + df_eph['PP3F_TOT']

estadisticas_horas = df_eph['horastrab'].describe(percentiles=[0.5])[['mean', 'std', 'min', '50%', 'max']]
estadisticas_horas.index = ['Promedio', 'Desviación estándar', 'Mínimo', 'Mediana (p50)', 'Máximo']

print("Estadísticas descriptivas de horastrab:\n")
print(estadisticas_horas)


Estadísticas descriptivas de horastrab:

Promedio                20.594300
Desviación estándar     23.866202
Mínimo                   0.000000
Mediana (p50)            8.000000
Máximo                 137.000000
Name: horastrab, dtype: float64


Luego de limpiar los valores correspondientes a "No sabe/No responde" (99, 999 y 9999), y analizando la base completa, tenemos como resultado un promedio de 20,6 horas trabajadas por semana, que sugiere una tendencia hacia la subocupación o trabajos de jornada parcial en la muestra. 
La desviación estándar de 23.9 horas indica una alta dispersión por lo que indica que hay un rango muy amplio de horas trabajadas.
El mínimo es 0, reflejando casos sin horas trabajadas. 
La mediana es de 8 horas.
El máximo de 137 horas es elevado, casi 20 horas por día, posible en situaciones de trabajo informal o múlitiples ocupaciones.

In [73]:
estad_2004 = df_eph[df_eph['ANO4'] == 2004]['horastrab'].describe(percentiles=[0.5])[['mean', 'std', 'min', '50%', 'max']]
print("Estadísticas descriptivas - Año 2004:")
print(estad_2004)

estad_2024 = df_eph[df_eph['ANO4'] == 2024]['horastrab'].describe(percentiles=[0.5])[['mean', 'std', 'min', '50%', 'max']]
print("\nEstadísticas descriptivas - Año 2024:")
print(estad_2024)


Estadísticas descriptivas - Año 2004:
mean     14.131987
std      22.961491
min       0.000000
50%       0.000000
max     137.000000
Name: horastrab, dtype: float64

Estadísticas descriptivas - Año 2024:
mean     35.002513
std      19.062046
min       0.000000
50%      40.000000
max     133.000000
Name: horastrab, dtype: float64


En 2004, tanto la media (14,1) como la mediana (0) indican que una gran parte de la muestra no trabajaba o trabajaba muy pocas horas. En cambio, en 2024, la mediana de 40 horas, muestra que la mayoría de las personas ocupadas trabajan jornada completa.
La desviación estándar es mayor en 2004, lo que refuerza la idea de una distribución más desigual (algunos trabajan muchas horas y muchos no trabajan nada). En 2024, la dispersión es más baja y centrada en 40hs.

In [74]:
# Filtrar por edad (15 a 65 años) y condición de PEA (ocupado o desocupado)
df_pea_edad = df_eph[(df_eph['CH06'] >= 15) & (df_eph['CH06'] <= 65) & (df_eph['ESTADO'].isin([1, 2]))]

pea_edad_2004 = df_pea_edad[df_pea_edad['ANO4'] == 2004]
pea_edad_2024 = df_pea_edad[df_pea_edad['ANO4'] == 2024]

estad_pea_2004 = pea_edad_2004['horastrab'].describe(percentiles=[0.5])[['mean', 'std', 'min', '50%', 'max']]
estad_pea_2024 = pea_edad_2024['horastrab'].describe(percentiles=[0.5])[['mean', 'std', 'min', '50%', 'max']]

print("Estadísticas descriptivas - PEA (15-65 años) 2004:\n", estad_pea_2004)
print("\nEstadísticas descriptivas - PEA (15-65 años) 2024:\n", estad_pea_2024)


Estadísticas descriptivas - PEA (15-65 años) 2004:
 mean     32.572354
std      24.851400
min       0.000000
50%      35.000000
max     137.000000
Name: horastrab, dtype: float64

Estadísticas descriptivas - PEA (15-65 años) 2024:
 mean     35.174711
std      19.018338
min       0.000000
50%      40.000000
max     133.000000
Name: horastrab, dtype: float64


Analizando los datos, tomando a la Población Económicamente Activa de 15 a 65 años, el promedio de horas trabajadas es 2.6 horas más en 2024 (32.6 a 35.2).
La mediana aumenta de 35 horas en 2004 a 40 horas en 2024, lo que sugiere que más personas en 2024 están trabajando a jornada completa.
La desviación estándar baja de 24.85 horas en 2004 a 19.02 en 2024, lo que indica que la distribución de las horas trabajadas en 2024 está más concentrada en torno a las 40 horas, y hay menos variabilidad en las horas trabajadas.
A pesar de la mayor concentración de jornadas cercanas a las 40 horas, los valores máximos (137 horas en 2004 y 133 horas en 2024), que son valores atípicos, puede reflejar trabajos con muchas horas adicionales, trabajo informal o personas con trabajos múltiples.

In [75]:
# Filtrar la base para la región NEA
df_nea = df_eph[df_eph['REGION'] == 41]

# Función resumen por año ajustada
def resumen_por_año(df, año):
    df_año = df[df['ANO4'] == año]
    total_obs = df_año.shape[0]
    nas_estado = (df_año['ESTADO'] == 0).sum()
    ocupados = (df_año['ESTADO'] == 1).sum()
    desocupados = (df_año['ESTADO'] == 2).sum()
    variables_limpias = df_año.dropna(axis=1).shape[1]
    return total_obs, nas_estado, ocupados, desocupados, variables_limpias

res_2004 = resumen_por_año(df_nea, 2004)
res_2024 = resumen_por_año(df_nea, 2024)

# Totales
total_obs = df_nea.shape[0]
nas_estado_total = (df_nea['ESTADO'] == 0).sum()
ocupados_total = (df_nea['ESTADO'] == 1).sum()
desocupados_total = (df_nea['ESTADO'] == 2).sum()
variables_limpias_total = df_nea.dropna(axis=1).shape[1]

# Tabla resumen
tabla = pd.DataFrame({
    '2004': res_2004,
    '2024': res_2024,
    'Total': [total_obs, nas_estado_total, ocupados_total, desocupados_total, variables_limpias_total]
}, index=[
    'Cantidad observaciones',
    'Cantidad de observaciones con Nas en la variable "Estado"',
    'Cantidad de Ocupados',
    'Cantidad de Desocupados',
    'Cantidad de variables limpias y homogeneizadas'
])

print(tabla)

                                                    2004  2024  Total
Cantidad observaciones                              5343  4850  10193
Cantidad de observaciones con Nas en la variabl...     5     6     11
Cantidad de Ocupados                                1789  2111   3900
Cantidad de Desocupados                              215   127    342
Cantidad de variables limpias y homogeneizadas       173    74     74


La base unificada para la región NEA cuenta con un total de 10193 observaciones, distribuidas entre 5343 en 2004 y 4850 en 2024. La cantidad de observaciones sin dato válido (Nas, que en la base está con el código 0) en la variable ESTADO es muy baja en ambos años, sumando apenas 11 casos en total, lo cual indica buena calidad de respuesta para esta variable. Respecto a la condición de actividad, en 2004 se registraron 1789 ocupados y 215 desocupados, mientras en el año 2024 los ocupados ascendieron a 2111 y los desocupados descendieron a 127, lo que podría estar reflejando cambios en el mercado laboral.